In [ ]:
import shutil
import pandas as pd
import openpyxl
from openpyxl import Workbook, worksheet
from openpyxl import load_workbook
from IPython.core.interactiveshell import InteractiveShell
import os

InteractiveShell.ast_node_interactivity = "all"

ipynb_path = os.path.dirname(os.path.realpath("__file__"))
print(ipynb_path)

original = r'C:\Users\Sofie.Dittmann\OneDrive - Lehman'"'"'s\Analysis\Pricing\SKU_classification_initial_list.xlsx'
target = r'C:\Users\Sofie.Dittmann\OneDrive - Lehman'"'"'s\Code\Python\Jupyter\Pricing\SKU_classification_initial_list.xlsx'

shutil.copyfile(original, target)

df_original = pd.read_excel('SKU_classification_initial_list.xlsx', engine='openpyxl')
#df_original.to_excel('2022_04_19 SKU classification initial list.xlsx')

work_book = load_workbook('SKU_classification_initial_list.xlsx', read_only = False)

work_sheet = work_book.active
work_sheet.freeze_panes = None

sheet_names = work_book.sheetnames

work_sheet = work_book[sheet_names[0]]

hidden_rows_idx = [
        row - 2 
        for row, dimension in work_sheet.row_dimensions.items() 
        if dimension.hidden
    ]

df_original = pd.read_excel('SKU_classification_initial_list.xlsx')

df_original = df_original.fillna('')

df_original.drop(['PUBLISHED', 'SYNCED', 'StdCost', 'LandedCost', 'BasePrice_Dt_Lst_Chg', 'BASEPRICE', 
                  'BASEMARGIN', 'WebPrice_Dt_Lst_Chg', 'CatalogPrice', 'CATMARGIN', 'WebPrice', 'MAP_Price', 
                  'Wiser.Price', 'Cost', 'MAP_Margin', 'Wiser_Margin', 'PickMargin', 'PriceNew'], axis = 1, inplace = True)

df_original_filtered_PriceChange = df_original[df_original['PriceChange'] == 'yes']

df_original_filtered_PriceChange = df_original_filtered_PriceChange.reset_index(drop=True)

import pyodbc 

cnxn = pyodbc.connect(driver = '{ODBC Driver 17 for SQL Server}', server = 'tcp:10.3.6.50', database = 'reports', trusted_connection = 'yes')

cursor = cnxn.cursor()

import pandas as pd

query = '''

SELECT *

FROM SD_repricing

'''

dfinventory = pd.read_sql(query, cnxn)

dfinventory_merge = df_original_filtered_PriceChange.merge(dfinventory, left_on=['ITEMID'], right_on=['ITEMID'], how="left")
#dfinventory_merge = df_original_filtered_PriceChange.merge(dfinventory, left_on=['ITEMID'], right_on=['ITEMID'], how="left", suffixes=("", "_right"))

cols = cols = [3,5,6,7,11,12,13,14,15,16]

dfinventory_merge.drop(dfinventory_merge.columns[cols], axis = 1, inplace = True)

dfinventory_merge.drop_duplicates(subset='ITEMID', inplace = True)

dfinventory_merge = dfinventory_merge.reset_index(drop=True)

dfinventory_merge.rename(columns={'NAMEALIAS_x':'NAMEALIAS','ItemGroup':'DEPT'})

dfinventory_merge.dropna(subset=['VEND_ID','Location','ITEMBARCODE','ExternalID'], how='all',inplace=True)

dfinventory_merge = dfinventory_merge.reset_index(drop=True)

dfinventory_merge